# Training on AzureML with Fastai
Because training on a rented VM is more expensive than sending to a low-priority (hence low cost) VM to train, especially for long-term model. This is created. 

Note that the folder directory is not full here. **If you don't want to make change to anything in this file**, make sure you have a folder in the same directory as this file called `to_launch`, and in that folder contains 4 items: `lm_model.py`, `nlputils.py` (with all necessary functions of course), `choice_7.npy`, and `counter_7.pkl`. 

In [19]:
from azureml.core import Workspace

ws = Workspace.from_config()

In [20]:
from azureml.core import Environment

fastai_env = Environment("fastai")
fastai_env.docker.enabled = True

'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [21]:
fastai_env.docker.base_image = "fastdotai/fastai:latest"
fastai_env.python.user_managed_dependencies = True

In [22]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "nc6sv2"  # Single P100
# cluster_name = "nc6sv3"  # Single V100

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='STANDARD_NC6S_V2',  # 'STANDARD_NC6S_V3' for V100. 
        max_nodes=1)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current AmlCompute
print(compute_target.get_status().serialize())

Found existing compute target.
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-11-22T15:13:41.277000+00:00', 'errors': None, 'creationTime': '2021-11-22T10:38:49.878319+00:00', 'modifiedTime': '2021-11-22T10:39:07.089797+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 1, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'LowPriority', 'vmSize': 'STANDARD_NC6S_V2'}


In [23]:
from azureml.core import ScriptRunConfig

fastai_conf = ScriptRunConfig(
    source_directory="to_launch",
    script="lm_model.py",
    compute_target=compute_target,
    environment=fastai_env
)

In [24]:
from azureml.core import Experiment

run = Experiment(ws, "fastai-custom-image").submit(fastai_conf)
run.wait_for_completion(show_output=True)

RunId: fastai-custom-image_1637597242_1b1778be
Web View: https://ml.azure.com/runs/fastai-custom-image_1637597242_1b1778be?wsid=/subscriptions/2699b46d-b293-4398-af19-3f902cf2d287/resourcegroups/mlwest2/workspaces/mlwest2&tid=ce1926b3-4f07-45ee-a67c-e02806b3f612
